In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import os
from skimage.io import imread, imshow, imread_collection
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#%matplotlib inline


In [ ]:
#reading the image
img_1 = imread('/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/train_resized/img_12.jpeg',as_gray=True)
imshow(img_1)
print(img_1.shape)

In [ ]:
# width height ratio width to height 1;2
# skimage  takes the input as height x width.
#resizing image 
resized_img_1 = resize(img_1, (128,64)) #64x128 default values of original file. 1:2 ratio important..
imshow(resized_img_1) 
print(resized_img_1.shape)

In [ ]:
#creating hog features , use default values..
fd_1, hog_image_1 = hog(resized_img_1, feature_vector=True, visualize=True)
fd_1.shape
#pixels_per_cell=3,3 --> 3218436, 8,8 -->443556

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 8), sharex=True, sharey=True) 
ax1.imshow(resized_img_1, cmap=plt.cm.gray) 
ax1.set_title('Input image') 
# Rescale histogram for better display 
hog_image_rescaled_1 = exposure.rescale_intensity(hog_image_1, in_range=(0, 10)) 
ax2.imshow(hog_image_rescaled_1, cmap=plt.cm.gray) 
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

In [ ]:
images = []
def load_images_from_folder(folder):
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),cv2.IMREAD_GRAYSCALE)
        if img is not None:
            #print(img.shape) #((2823, 2823, 3))
            images.append(img)
    #return images

load_images_from_folder("/content/drive/Shared drives/CSCE 633(Fall 2020) HW5/train_resized")

In [ ]:
len(images)

In [ ]:
# skimage  takes the input as height x width.
#resizing image 
resized_images=[]
for i in images:
  r_im= resize(i, (256,128)) 
  resized_images.append(r_im)

In [ ]:
len(resized_images)

In [ ]:
fd_list=[]
hog_images=[]
for imge in resized_images:
  fd_i, hog_image = hog(imge, feature_vector=True, visualize=True)
  fd_list.append(fd_i)
  hog_images.append(hog_image)

print(hog_image.shape)
print(fd_i) #one image as a feacture vectore array

In [ ]:
print(len(hog_images))
print(len(fd_list))
print(fd_i.shape)

In [ ]:
#save fdlist into csv file
import pandas as pd 
import csv
resultFile = open("fd_list.csv",'w')
wr = csv.writer(resultFile)
for i in fd_list:
    wr.writerows([i])

In [ ]:
#can I make a new data to see what is happenening
dataframe = pd.read_csv('/content/drive/Shared drives/CSCE 633(Fall 2020) HW5/train.csv')
dataframe.head(3)
dataframe.tail(3)

In [ ]:
X_traincsv = dataframe.iloc[:, 1:-1].values #filename did not added
y_traincsv = dataframe.iloc[:, -1].values
print(X_traincsv.shape)
print(y_traincsv.shape)

In [ ]:
np.shape(fd_list)


In [ ]:
# after hog function i save the results as csv file.
#read from there..
columns = pd.read_csv('/content/fd_list.csv',header=None)
new_values = columns.iloc[:, 1:].values
print(new_values.shape)

In [ ]:
# check new values(afterhog) and real csv file label column
print(new_values.shape)
print(y_traincsv.shape)

In [ ]:
# create a new structure with these 2
result= np.concatenate((new_values, y_traincsv.reshape(len(y_traincsv),1)), axis=1)
print(result.shape)

In [ ]:
#save new structure into csv file..
import pandas as pd 
import csv
resultwithlabel = open("resultwithlabel.csv",'w')
wr = csv.writer(resultwithlabel)
for i in result:
    wr.writerows([i])

In [ ]:
newDataframe = pd.read_csv('/content/resultwithlabel.csv',header=None)
checkvalues = newDataframe.iloc[:, 1:].values
checkresult = newDataframe.iloc[:, -1].values
print(checkvalues.shape)
print(checkresult.shape)
newDataframe.head(3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(checkvalues, checkresult, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 5)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
print(X_train.shape)
print(X_test.shape)

# TRY MODELS

## RandomForestClassifier ENTROPY

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifierentropy = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifierentropy.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifierentropy.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

## RandomForestClassifier GINI

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifiergini = RandomForestClassifier(n_estimators = 10, criterion = 'gini', random_state = 0)
classifiergini.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifiergini.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

## SVC sigmoid

In [ ]:
from sklearn.svm import SVC
classifiersvc = SVC(kernel='sigmoid', random_state = 0)
classifiersvc.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifiersvc.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

# LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
classifierlr = LogisticRegression(random_state = 0)
classifierlr.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifierlr.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)